In [ ]:
from taichi_em_discovery import EMDiscovery
import numpy as np

B, N = 100_000, 64
dt, T = 0.1, 1000

sim = EMDiscovery(B=B, N=N, dt=dt, T=T, max_abs_a=8.0, beta_ema=0.1, seed=1)
sim.run()

K = sim.K_numpy()
activity, ema, flips, overflow, steps = sim.metrics_numpy()
idx, score = EMDiscovery.select_interesting(
    K, activity, ema, flips, overflow,
    eps_converge=2e-4, lambda_flip=0.05, top_k=1000
)

A = sim.state_numpy()  # optional
K_sel = K[idx]
A_sel = A[idx]
print("Selected:", len(idx), "Top score:", score[idx[0]])
print("n_overflow:", overflow.sum(), "/", B)


In [ ]:
import numpy as np
from taichi_em_snapshot_divergence import EMSnapshotDivergence

# --- run the sim ---
B, N = 50_000, 64
dt, T, K_snap = 0.1, 1000, 10
sim = EMSnapshotDivergence(B=B, N=N, dt=dt, T=T, K_snap=K_snap, max_abs_a=2, seed=0)
sim.run()

# --- retrieve raw outputs ---
K = sim.K_numpy()                 # (B, 5)  -> θ per EM
overflow = sim.overflow_numpy()   # (B,)    -> 0/1 flags
snaps = sim.snapshots_numpy()     # (S, B, N)
scores = EMSnapshotDivergence.compute_snapshot_divergence(
    snaps, lag_choices=(1,2,3,4), seed=0
)                                  # (B,)

# --- mask out overflowed, then rank ---
valid = (overflow == 0)
scores_valid = np.where(valid, scores, -np.inf)
order = np.argsort(-scores_valid)        # descending
top_k = 1000
idx = order[:top_k]                      # indexes into the original batch 0..B-1

# --- pick things for those indexes ---
theta_top = K[idx]                       # (top_k, 5)
scores_top = scores[idx]                 # (top_k,)
last_snapshot_top = snaps[-1, idx, :]    # (top_k, N) final states (optional)
all_snaps_top = snaps[:, idx, :]         # (S, top_k, N) all snapshots (optional)

# --- (optional) save for later ---
np.savez("em_discovery_topK.npz",
         idx=idx, theta=theta_top, scores=scores_top,
         overflow=overflow[idx],
         last_snapshot=last_snapshot_top)


In [ ]:
test=theta_top[10,:]

In [ ]:
from em_numpy_single import simulate_single_em
import matplotlib.pyplot as plt

#K_test=theta_top[0]
K_test= np.array([-0.28755745,  0.02690542,  0.26518247,0,0 ])
print("k_test",K_test)

# Pick one of the top-K params from the Taichi discovery pass
K_star = K_test.astype(np.float32)  # example

a_final, overflow, traj = simulate_single_em(
    K_star,
    N=64,
    dt=0.1,
    T=500,
    max_abs_a=5.0,
    seed=1,
    A0=None,           # or pass a custom initial state (N,)
    return_traj=True,  # set False if you only need the last state
)

print("overflow:", overflow, "final shape:", a_final.shape)

#init
plt.figure(figsize=(15, 15))
plt.imshow(traj[:200,:])
plt.colorbar()
plt.show()

# if traj is not None: plot or inspect it: shape (T+1, N)
plt.figure(figsize=(20, 20))
plt.imshow(traj[-300:,:])
plt.colorbar()
plt.show()

# if traj is not None: plot or inspect it: shape (T+1, N)
plt.figure(figsize=(200, 200))
plt.imshow(traj[:,:])
plt.colorbar()
plt.show()

In [ ]:
from taichi_em_snapshot_divergence import EMSnapshotDivergence
import numpy as np

B, N = 300_000, 15
dt, T, K_snap = 0.1, 2000, 10
sim = EMSnapshotDivergence(B=B, N=N, dt=dt, T=T, K_snap=K_snap, max_abs_a=3.0, seed=1)
sim.run()

snaps = sim.snapshots_numpy()                   # (num_snaps, B, N) from t0..T
K = sim.K_numpy()                               # (B, 5)
overflow=sim.overflow_numpy()
scores, idx_sorted = EMSnapshotDivergence.compute_snapshot_divergence(snaps, seed=0, lag_choices=(1,2,3,7,10))

top_k = 50
idx = idx_sorted[:top_k]
theta_top = K[idx]
scores_top = scores[idx]
print("best_score",scores_top[0])
print("best_theta",theta_top[0])
print("n_overflow:", overflow.sum(), "/", B)

In [ ]:
theta_top[0]

In [ ]:
out_csv = "em_snapshot_divergence_results.csv"
write_header = not os.path.exists(out_csv)

seeds = [0, 1, 2, 3, 4]
B, N = 4096, 64
dt, T, K_snap = 0.05, 320, 20
top_k = 100

with open(out_csv, "a", newline="") as f:
    w = csv.writer(f)
    if write_header:
        w.writerow(["seed", "em_idx", "score", "k1", "k2", "k3", "k4", "k5"])

    for seed in seeds:
        sim = EMSnapshotDivergence(B=B, N=N, dt=dt, T=T, K_snap=K_snap, max_abs_a=5.0, seed=seed)
        sim.run()
        snaps = sim.snapshots_numpy()
        K = sim.K_numpy()
        scores, idx_sorted = EMSnapshotDivergence.compute_snapshot_divergence(
            snaps, lag_choices=(1,2,3), seed=seed, return_sorted_idx=True
        )
        sel = idx_sorted[:top_k]
        for em_idx in sel:
            kvec = K[em_idx]
            w.writerow([seed, int(em_idx), float(scores[em_idx]),
                        float(kvec[0]), float(kvec[1]), float(kvec[2]), float(kvec[3]), float(kvec[4])])

print(f"Appended results to {out_csv}")